In [2]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
import sys

# --- 1. PREPARACIÓN DE ENTORNO Y RUTAS ---

# Montar Google Drive (se ejecuta siempre para asegurar la conexión)
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"Error al montar Google Drive: {e}")
    sys.exit()

# Rutas de archivos y carpetas
TRANSCRIPCIONES_PATH = "/content/drive/MyDrive/TFG_2025/Entrevistas/"
FISIO_DIR_PATH = "/content/drive/MyDrive/TFG_2025/datos_post_experimentos/"
FISIO_FILE_NAME = "resultados_consolidados.csv"
FISIO_FULL_PATH = os.path.join(FISIO_DIR_PATH, FISIO_FILE_NAME)

QUAL_PATH = "/content/drive/MyDrive/TFG_2025/Conteo_Conceptos_Participante.csv"

# Descargar y definir stopwords de forma robusta
try:
    nltk.download('stopwords', quiet=True)
    stop_words_es = set(stopwords.words('spanish'))
except:
    # Usamos la lista codificada como respaldo
    stop_words_es = {'un', 'una', 'unas', 'unos', 'uno', 'sobre', 'también', 'tras', 'otro', 'otras', 'otros', 'otra', 'de', 'la', 'lo', 'las', 'los', 'a', 'ante', 'bajo', 'cabe', 'con', 'contra', 'desde', 'durante', 'en', 'entre', 'hacia', 'hasta', 'mediante', 'para', 'por', 'según', 'sin', 'so', 'sobre', 'tras', 'o', 'u', 'el', 'al', 'del', 'mi', 'mis', 'tu', 'tus', 'su', 'sus', 'aquel', 'aquella', 'aquellos', 'aquellas', 'este', 'esta', 'estos', 'estas', 'es', 'sea', 'pero', 'mas', 'más', 'y', 'e', 'si', 'sino', 'sólo', 'solo', 'tal', 'vez', 'toda', 'todo', 'todos', 'todas', 'qué', 'cual', 'cuales', 'quien', 'quienes', 'yo', 'me', 'mi', 'mío', 'mía', 'míos', 'mías', 'tú', 'te', 'ti', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'él', 'ella', 'ello', 'nos', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vos', 'vosotros', 'vosotras', 'vuestro', 'vuestra', 'vuestros', 'vuestras'}


# Diccionario de conceptos clave para el TFG
conceptos_clave = {
    'Estres_Ansiedad': ['estrés', 'ansiedad', 'nervioso', 'agobio', 'presión', 'malo', 'difícil'],
    'Recuperacion_Calma': ['calma', 'relajación', 'tranquilidad', 'paz', 'descanso', 'mejor'],
    'Metacognicion_Reflexion': ['sentir', 'pensar', 'darse', 'cuenta', 'conciencia', 'reflejar', 'mente', 'proceso'],
    'Intervencion_Sonido': ['sonido', 'música', 'paisaje', 'audio'],
}


def clean_and_tokenize(text):
    # Tokenización y limpieza
    tokens = re.findall(r'[a-zA-ZáéíóúüñÁÉÍÓÚÜÑ]+', text.lower())
    tokens = [word for word in tokens if word not in stop_words_es]
    tokens = [word for word in tokens if word not in ('gracias', 'sí', 'no', 'vale', 'entonces', 'pues', 'claro', 'eh', 'e', 'o')]
    return tokens

# --- 2. CREACIÓN DE BASE DE DATOS CUALITATIVA (para asegurar el archivo) ---

df_conteo = []

try:
    for filename in os.listdir(TRANSCRIPCIONES_PATH):
        if filename.endswith(".txt"):
            ruta_archivo = os.path.join(TRANSCRIPCIONES_PATH, filename)
            with open(ruta_archivo, 'r', encoding='utf-8') as f:
                content = f.read()

            tokens = clean_and_tokenize(content)
            word_counts = Counter(tokens)

            participante_data = {'ID_Participante': filename.split('.')[0]}

            for concepto, palabras in conceptos_clave.items():
                count = sum(word_counts[palabra] for palabra in palabras)
                participante_data[concepto] = count

            df_conteo.append(participante_data)

    df_qualitativo = pd.DataFrame(df_conteo)
    df_qualitativo.to_csv(QUAL_PATH, index=False)
    print("Base de datos cualitativa (conteo) creada con éxito.")

except FileNotFoundError:
    # Este error se debe a la ruta de Drive, la manejamos aquí y terminamos elegantemente.
    print(f"ERROR FATAL: No se pudieron leer las transcripciones en {TRANSCRIPCIONES_PATH}. Revise la ruta de su Drive.")
    sys.exit()

# --- 3. CÁLCULO DE CORRELACIÓN INTERDISCIPLINARIA ---

# Cargar datos cualitativos (Conteo de Palabras)
df_qual = df_qualitativo.copy()
df_qual.rename(columns={'ID_Participante': 'participantId'}, inplace=True)

# Cargar datos cuantitativos (Fisiológicos)
try:
    df_fisio = pd.read_csv(FISIO_FULL_PATH)
except FileNotFoundError:
    print(f"ERROR FATAL: Archivo fisiológico no encontrado en {FISIO_FULL_PATH}. Revise la ruta.")
    sys.exit()


# Asegurar que las columnas de ID sean del mismo tipo (string)
df_qual['participantId'] = df_qual['participantId'].astype(str)
df_fisio['participantId'] = df_fisio['participantId'].astype(str)

# FUSIONAR LOS DATASETS
df_merged = pd.merge(df_qual, df_fisio, on='participantId', how='inner')

print(f"\nParticipantes fusionados para análisis: {len(df_merged)}")

# Definir las variables para el cruce:
qual_vars = list(conceptos_clave.keys())
fisio_vars = ['gsr_change_relajacion', 'arousal_final', 'valence_final', 'mean_stroop_pc']

correlations = []

for q_var in qual_vars:
    for f_var in fisio_vars:
        # Calcular la correlación de Pearson (r)
        correlation = df_merged[[q_var, f_var]].corr(method='pearson').iloc[0, 1]
        n_obs = df_merged[[q_var, f_var]].dropna().shape[0]

        correlations.append({
            'Variable_Qualitativa': q_var,
            'Variable_Fisiologica': f_var,
            'Correlacion_r': correlation,
            'Observaciones_n': n_obs
        })

# CREAR LA TABLA DE RESULTADOS DE CORRELACIÓN
df_corr = pd.DataFrame(correlations)
df_corr['abs_r'] = df_corr['Correlacion_r'].abs()
df_corr = df_corr.sort_values(by='abs_r', ascending=False).drop(columns=['abs_r'])

# --- 4. RESULTADOS FINALES ---

# Guardar el resultado final en Drive
OUTPUT_CORR_PATH = "/content/drive/MyDrive/TFG_2025/Tabla_Correlacion_TFG.csv"
df_corr.to_csv(OUTPUT_CORR_PATH, index=False)

print("\n--- TABLA DE CORRELACIONES INTERDISCIPLINARIAS ---")
print("Las correlaciones más cercanas a 1.0 (positivas) o -1.0 (negativas) son las más fuertes.")
print(df_corr.round(3).to_markdown(index=False))
print(f"\nResultados de correlación guardados en Drive: {OUTPUT_CORR_PATH} 🎉")

Mounted at /content/drive
Base de datos cualitativa (conteo) creada con éxito.

Participantes fusionados para análisis: 14

--- TABLA DE CORRELACIONES INTERDISCIPLINARIAS ---
Las correlaciones más cercanas a 1.0 (positivas) o -1.0 (negativas) son las más fuertes.
| Variable_Qualitativa    | Variable_Fisiologica   |   Correlacion_r |   Observaciones_n |
|:------------------------|:-----------------------|----------------:|------------------:|
| Metacognicion_Reflexion | valence_final          |           0.45  |                14 |
| Estres_Ansiedad         | arousal_final          |           0.396 |                14 |
| Recuperacion_Calma      | mean_stroop_pc         |          -0.35  |                14 |
| Estres_Ansiedad         | mean_stroop_pc         |          -0.339 |                14 |
| Estres_Ansiedad         | gsr_change_relajacion  |           0.337 |                14 |
| Recuperacion_Calma      | valence_final          |           0.289 |                14 |
| Interv